In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path
import tensorflow as tf
import tensorflow_addons as tfa
import PIL
import PIL.Image
from random import randint
from pickle import dump, load

In [3]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.7, val_split=0.2, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)


    return train_ds, val_ds, test_ds

def random_brightness(image):
    return tf.image.random_brightness(image, max_delta=0.2)

def build_metrics(no_of_classes):
    metrics = []
    metrics.append('accuracy')
    for i in range(0, no_of_classes):
        metric_name = '{}{}'.format('precision', i)
        metrics.append(tf.keras.metrics.Precision(class_id=i, name=metric_name))

    for i in range(0, no_of_classes):
        metric_name = '{}{}'.format('recall', i)
        metrics.append(tf.keras.metrics.Recall(class_id=i, name=metric_name))

    metrics.append(tf.keras.metrics.AUC(curve='ROC', name='roc_auc', multi_label=True, num_labels=no_of_classes))
    metrics.append(tfa.metrics.MultiLabelConfusionMatrix(num_classes=no_of_classes))
    return metrics

def get_precision_evaluation(history):
    precision = 0.0
    precision_begin = 2
    precision_end = precision_begin + num_classes
    for i in range(precision_begin, precision_end):
        precision += history[i]

    return precision / num_classes

def get_recall_evaluation(history):
    recall = 0.0
    recall_begin = 2 + num_classes
    recall_end = recall_begin + num_classes
    for i in range(recall_begin, recall_end):
        recall += history[i]

    return recall / num_classes

def get_f1_score(precision, recall):
    f1_score = 0.0
    if (precision + recall) != 0:
        f1_score = (2 * precision * recall) / (precision + recall)

    return f1_score

def get_precision_histories(history):
    precision_history = [0] * initial_epochs
    val_precision_history = [0] * initial_epochs
    for i in range(0, num_classes):
        metric_name = '{}{}'.format('precision', i)
        val_metric_name = 'val_' + metric_name
        precision_history = [sum(x) for x in zip(precision_history, history[metric_name])]
        val_precision_history = [sum(x) for x in zip(precision_history, history[val_metric_name])]

    for i in range(0, initial_epochs):
        precision_history[i] = precision_history[i] / num_classes
        val_precision_history[i] = val_precision_history[i] / num_classes

    return precision_history, val_precision_history

def get_recall_histories(history):
    recall_history = [0] * initial_epochs
    val_recall_history = [0] * initial_epochs
    for i in range(0, num_classes):
        metric_name = '{}{}'.format('recall', i)
        val_metric_name = 'val_' + metric_name
        recall_history = [sum(x) for x in zip(recall_history, history[metric_name])]
        val_recall_history = [sum(x) for x in zip(recall_history, history[val_metric_name])]

    for i in range(0, initial_epochs):
        recall_history[i] = recall_history[i] / num_classes
        val_recall_history[i] = val_recall_history[i] / num_classes

    return recall_history, val_recall_history

def calculate_f1_scores(precision, recall, val_precision, val_recall):
    f1_score_temp = np.empty(len(precision))
    val_f1_score_temp = np.empty(len(f1_score_temp))
    for i in range(len(precision)):
        harmonic_avg = 0.0
        if (precision[i] + recall[i]) != 0:
            harmonic_avg = (2 * precision[i] * recall[i]) / (precision[i] + recall[i])

        val_harmonic_avg = 0.0
        if (val_precision[i] + val_recall [i]) != 0:
            val_harmonic_avg = (2 * val_precision[i] * val_recall [i]) / (val_precision[i] + val_recall [i])

        np.put(f1_score_temp, i, harmonic_avg)
        np.put(val_f1_score_temp, i, val_harmonic_avg)
    return [f1_score_temp, val_f1_score_temp]

def combine_2_histories(history1, history2):
    combined_history = {}
    for key in history1.history.keys():
        combined_history[key] = history1.history[key] + history2.history[key]
    return combined_history

def combine_multiple_histories(histories):
    combined_history = {}
    for history in histories:
        for key, value in history.items():
            if key not in combined_history:
                combined_history[key] = value.copy()
            else:
                combined_history[key].extend(value.copy())
    return combined_history

In [4]:
PATH = "D:\y3\licenta_shit\CCSN_v2"
file_types = ('*.jpg', '*.jpeg', '*.png')

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

dataset = tf.keras.preprocessing.image_dataset_from_directory(PATH,
                                      shuffle=True,
                                      label_mode='categorical',
                                      batch_size=BATCH_SIZE,
                                      image_size=IMG_SIZE)

class_names = dataset.class_names
num_classes = len(class_names)
print(num_classes)
print(class_names)

train_dataset, validation_dataset, test_dataset = get_dataset_partitions_tf(dataset, ds_size=len(dataset))

print('Size of training dataset batch: %d' % len(train_dataset))
print('Size of validation dataset batch: %d' % len(validation_dataset))
print('Size of test dataset batch: %d' % len(test_dataset))

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation((-0.2,0.3)),
    tf.keras.layers.RandomContrast(0.2, 0.5),
    tf.keras.layers.Lambda(random_brightness),
    tf.keras.layers.GaussianNoise(0.1)
])

base_model = tf.keras.applications.EfficientNetV2B0(input_shape=IMG_SHAPE, include_top=False, weights=None)

image_batch, label_batch = train_dataset.as_numpy_iterator().next()
feature_batch = base_model(image_batch)

base_model.trainable = True

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

Found 2543 files belonging to 11 classes.
11
['Ac', 'As', 'Cb', 'Cc', 'Ci', 'Cs', 'Ct', 'Cu', 'Ns', 'Sc', 'St']
Size of training dataset batch: 56
Size of validation dataset batch: 16
Size of test dataset batch: 8




In [5]:
metrics = build_metrics(num_classes)
history_array = []

In [6]:
base_learning_rate = 0.00001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=metrics)

In [7]:
initial_history = model.evaluate(validation_dataset)

initial_precision = get_precision_evaluation(initial_history)
initial_recall = get_recall_evaluation(initial_history)
initial_f1_score = get_f1_score(initial_precision, initial_recall)
initial_roc_auc_index = 2 + (2 * num_classes)

print("Initial loss: {:.2f}".format(initial_history[0]))
print("Initial accuracy: {:.2f}".format(initial_history[1]))
print("Initial precision: {:.2f}".format(initial_precision))
print("Initial recall: {:.2f}".format(initial_recall))
print("Initial F1 score: {:.2f}".format(initial_f1_score))
print("Initial ROC: {:.2f}".format(initial_history[initial_roc_auc_index]))



16/16 [==============================] - 7s 243ms/step - loss: 2.3975 - accuracy: 0.1484 - precision0: 0.0000e+00 - precision1: 0.0000e+00 - precision2: 0.0000e+00 - precision3: 0.0000e+00 - precision4: 0.0000e+00 - precision5: 0.0000e+00 - precision6: 0.0000e+00 - precision7: 0.0000e+00 - precision8: 0.0000e+00 - precision9: 0.0000e+00 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.0000e+00 - recall2: 0.0000e+00 - recall3: 0.0000e+00 - recall4: 0.0000e+00 - recall5: 0.0000e+00 - recall6: 0.0000e+00 - recall7: 0.0000e+00 - recall8: 0.0000e+00 - recall9: 0.0000e+00 - recall10: 0.0000e+00 - roc_auc: 0.5108 - Multilabel_confusion_matrix: 128.0000
Initial loss: 2.40
Initial accuracy: 0.15
Initial precision: 0.00
Initial recall: 0.00
Initial F1 score: 0.00
Initial ROC: 0.51


In [8]:
history25e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_25')
history_array.append(history25e.history)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_25.json', 'wb') as file:
    dump(history25e.history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #25 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 87s 1s/step - loss: 2.4428 - accuracy: 0.0960 - precision0: 0.0000e+00 - precision1: 0.0000e+00 - precision2: 0.0000e+00 - precision3: 0.0000e+00 - precision4: 0.0000e+00 - precision5: 0.1667 - precision6: 0.0909 - precision7: 0.0000e+00 - precision8: 0.0000e+00 - precision9: 0.0000e+00 - precision10: 1.0000 - recall0: 0.0000e+00 - recall1: 0.0000e+00 - recall2: 0.0000e+00 - recall3: 0.0000e+00 - recall4: 0.0000e+00 - recall5: 0.0098 - recall6: 0.0072 - recall7: 0.0000e+00 - recall8: 0.0000e+00 - recall9: 0.0000e+00 - recall10: 0.0074 - roc_auc: 0.5497 - Multilabel_confusion_matrix: 448.0000 - val_loss: 2.4296 - val_accuracy: 0.0918 - val_precision0: 0.0000e+00 - val_precision1: 0.0000e+00 - val_precision2: 0.0000e+00 - val_precision3: 0.0000e+00 - val_precision4: 0.0000e+00 - val_precision5: 0.5000 - val_precision6: 0.2000 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.0000e+00 - val_precision10: 0.0000

In [9]:
history50e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_50')
history_array.append(history50e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_50.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #50 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 88s 2s/step - loss: 2.1548 - accuracy: 0.2333 - precision0: 0.0000e+00 - precision1: 0.5000 - precision2: 0.0000e+00 - precision3: 0.0000e+00 - precision4: 0.0000e+00 - precision5: 0.0000e+00 - precision6: 0.6250 - precision7: 0.0000e+00 - precision8: 0.0000e+00 - precision9: 0.2857 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.0698 - recall2: 0.0000e+00 - recall3: 0.0000e+00 - recall4: 0.0000e+00 - recall5: 0.0000e+00 - recall6: 0.1064 - recall7: 0.0000e+00 - recall8: 0.0000e+00 - recall9: 0.0084 - recall10: 0.0000e+00 - roc_auc: 0.6936 - Multilabel_confusion_matrix: 448.0000 - val_loss: 2.1056 - val_accuracy: 0.2768 - val_precision0: 0.0000e+00 - val_precision1: 0.5556 - val_precision2: 0.0000e+00 - val_precision3: 1.0000 - val_precision4: 0.0000e+00 - val_precision5: 0.0000e+00 - val_precision6: 1.0000 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.0000e+00 - val_precision10: 0.0000e+00

In [10]:
history75e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_75')
history_array.append(history75e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_75.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #75 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 83s 1s/step - loss: 2.0197 - accuracy: 0.2851 - precision0: 0.0000e+00 - precision1: 0.6087 - precision2: 0.3333 - precision3: 1.0000 - precision4: 0.0000e+00 - precision5: 0.3000 - precision6: 0.7576 - precision7: 0.0000e+00 - precision8: 0.0000e+00 - precision9: 0.4250 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.1085 - recall2: 0.0119 - recall3: 0.0165 - recall4: 0.0000e+00 - recall5: 0.0146 - recall6: 0.1812 - recall7: 0.0000e+00 - recall8: 0.0000e+00 - recall9: 0.0708 - recall10: 0.0000e+00 - roc_auc: 0.7428 - Multilabel_confusion_matrix: 443.7500 - val_loss: 2.0132 - val_accuracy: 0.2969 - val_precision0: 0.0000e+00 - val_precision1: 0.1429 - val_precision2: 1.0000 - val_precision3: 0.0000e+00 - val_precision4: 0.0000e+00 - val_precision5: 1.0000 - val_precision6: 0.6667 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.6923 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val

In [11]:
history100e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_100')
history_array.append(history100e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_100.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #100 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 89s 2s/step - loss: 1.9759 - accuracy: 0.3059 - precision0: 0.0000e+00 - precision1: 0.6111 - precision2: 0.6364 - precision3: 0.3333 - precision4: 0.0000e+00 - precision5: 0.8000 - precision6: 0.7143 - precision7: 0.0000e+00 - precision8: 0.0000e+00 - precision9: 0.5741 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.0846 - recall2: 0.0446 - recall3: 0.0102 - recall4: 0.0000e+00 - recall5: 0.0206 - recall6: 0.2349 - recall7: 0.0000e+00 - recall8: 0.0000e+00 - recall9: 0.1372 - recall10: 0.0000e+00 - roc_auc: 0.7561 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.8559 - val_accuracy: 0.3457 - val_precision0: 0.0000e+00 - val_precision1: 0.7500 - val_precision2: 0.4286 - val_precision3: 0.0000e+00 - val_precision4: 0.0000e+00 - val_precision5: 1.0000 - val_precision6: 1.0000 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.7083 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val

In [12]:
history125e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_125')
history_array.append(history125e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_125.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #125 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 83s 1s/step - loss: 1.9012 - accuracy: 0.3341 - precision0: 0.0000e+00 - precision1: 0.6364 - precision2: 0.6522 - precision3: 0.5625 - precision4: 0.0000e+00 - precision5: 0.2500 - precision6: 0.7258 - precision7: 0.0000e+00 - precision8: 0.0000e+00 - precision9: 0.5833 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.1176 - recall2: 0.0926 - recall3: 0.0452 - recall4: 0.0000e+00 - recall5: 0.0095 - recall6: 0.3169 - recall7: 0.0000e+00 - recall8: 0.0000e+00 - recall9: 0.1842 - recall10: 0.0000e+00 - roc_auc: 0.7803 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.7522 - val_accuracy: 0.3960 - val_precision0: 0.0000e+00 - val_precision1: 0.8571 - val_precision2: 0.6923 - val_precision3: 0.1667 - val_precision4: 0.0000e+00 - val_precision5: 1.0000 - val_precision6: 0.7619 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.5862 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val_rec

In [13]:
history150e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_150')
history_array.append(history150e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_150.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #150 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 86s 2s/step - loss: 1.8699 - accuracy: 0.3544 - precision0: 0.0000e+00 - precision1: 0.5000 - precision2: 0.5714 - precision3: 0.3333 - precision4: 0.0000e+00 - precision5: 0.8333 - precision6: 0.7353 - precision7: 0.0000e+00 - precision8: 0.5000 - precision9: 0.6232 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.0827 - recall2: 0.0982 - recall3: 0.0435 - recall4: 0.0000e+00 - recall5: 0.0245 - recall6: 0.3472 - recall7: 0.0000e+00 - recall8: 0.0209 - recall9: 0.1838 - recall10: 0.0000e+00 - roc_auc: 0.7883 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.6857 - val_accuracy: 0.4199 - val_precision0: 0.0000e+00 - val_precision1: 0.9231 - val_precision2: 0.5600 - val_precision3: 0.3333 - val_precision4: 0.0000e+00 - val_precision5: 1.0000 - val_precision6: 0.7273 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.6562 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val_recall1: 0.

In [14]:
history175e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_175')
history_array.append(history175e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_175.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #175 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 90s 2s/step - loss: 1.8519 - accuracy: 0.3555 - precision0: 0.0000e+00 - precision1: 0.5185 - precision2: 0.5652 - precision3: 0.4412 - precision4: 0.0000e+00 - precision5: 0.5294 - precision6: 0.7612 - precision7: 0.0000e+00 - precision8: 0.3750 - precision9: 0.4615 - precision10: 0.0000e+00 - recall0: 0.0000e+00 - recall1: 0.1138 - recall2: 0.1595 - recall3: 0.0781 - recall4: 0.0000e+00 - recall5: 0.0437 - recall6: 0.3696 - recall7: 0.0000e+00 - recall8: 0.0308 - recall9: 0.1469 - recall10: 0.0000e+00 - roc_auc: 0.7898 - Multilabel_confusion_matrix: 448.0000 - val_loss: 1.6971 - val_accuracy: 0.4062 - val_precision0: 0.0000e+00 - val_precision1: 0.6667 - val_precision2: 0.6176 - val_precision3: 0.4444 - val_precision4: 0.0000e+00 - val_precision5: 0.5000 - val_precision6: 0.7600 - val_precision7: 0.0000e+00 - val_precision8: 0.0000e+00 - val_precision9: 0.5510 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val_recall1: 0.

In [15]:
history200e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_200')
history_array.append(history200e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_200.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #200 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 88s 2s/step - loss: 1.7782 - accuracy: 0.3666 - precision0: 0.5000 - precision1: 0.6400 - precision2: 0.5763 - precision3: 0.5294 - precision4: 0.0000e+00 - precision5: 0.5714 - precision6: 0.7846 - precision7: 0.0000e+00 - precision8: 0.6207 - precision9: 0.5566 - precision10: 0.0000e+00 - recall0: 0.0065 - recall1: 0.1203 - recall2: 0.2000 - recall3: 0.0978 - recall4: 0.0000e+00 - recall5: 0.0398 - recall6: 0.3723 - recall7: 0.0000e+00 - recall8: 0.0923 - recall9: 0.2438 - recall10: 0.0000e+00 - roc_auc: 0.8148 - Multilabel_confusion_matrix: 448.0000 - val_loss: 1.6865 - val_accuracy: 0.3945 - val_precision0: 0.0000e+00 - val_precision1: 0.8750 - val_precision2: 0.6829 - val_precision3: 0.6471 - val_precision4: 0.0000e+00 - val_precision5: 0.5000 - val_precision6: 0.7727 - val_precision7: 0.0000e+00 - val_precision8: 0.5000 - val_precision9: 0.5909 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val_recall1: 0.1628 - val_r

In [16]:
history225e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_225')
history_array.append(history225e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_225.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #225 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 87s 2s/step - loss: 1.7635 - accuracy: 0.3730 - precision0: 0.3333 - precision1: 0.6774 - precision2: 0.6909 - precision3: 0.5122 - precision4: 0.0000e+00 - precision5: 0.5333 - precision6: 0.7436 - precision7: 0.5000 - precision8: 0.4828 - precision9: 0.5888 - precision10: 0.0000e+00 - recall0: 0.0063 - recall1: 0.1567 - recall2: 0.2249 - recall3: 0.1173 - recall4: 0.0000e+00 - recall5: 0.0385 - recall6: 0.4265 - recall7: 0.0154 - recall8: 0.0757 - recall9: 0.2540 - recall10: 0.0000e+00 - roc_auc: 0.8150 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.6178 - val_accuracy: 0.4202 - val_precision0: 0.6667 - val_precision1: 0.8333 - val_precision2: 0.6250 - val_precision3: 0.4737 - val_precision4: 1.0000 - val_precision5: 0.7500 - val_precision6: 0.6571 - val_precision7: 1.0000 - val_precision8: 0.8333 - val_precision9: 0.6471 - val_precision10: 0.0000e+00 - val_recall0: 0.0377 - val_recall1: 0.1667 - val_recall2: 0.3947 - val_rec

In [17]:
history250e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_250')
history_array.append(history250e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_250.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #250 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 86s 2s/step - loss: 1.7313 - accuracy: 0.3938 - precision0: 0.6667 - precision1: 0.7188 - precision2: 0.6333 - precision3: 0.5472 - precision4: 1.0000 - precision5: 0.5000 - precision6: 0.6988 - precision7: 0.4000 - precision8: 0.5278 - precision9: 0.6304 - precision10: 0.0000e+00 - recall0: 0.0125 - recall1: 0.1949 - recall2: 0.2289 - recall3: 0.1518 - recall4: 0.0377 - recall5: 0.0265 - recall6: 0.4173 - recall7: 0.0164 - recall8: 0.0995 - recall9: 0.2427 - recall10: 0.0000e+00 - roc_auc: 0.8227 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.6545 - val_accuracy: 0.4355 - val_precision0: 0.0000e+00 - val_precision1: 0.6667 - val_precision2: 0.6389 - val_precision3: 0.5385 - val_precision4: 0.0000e+00 - val_precision5: 0.6667 - val_precision6: 0.7273 - val_precision7: 1.0000 - val_precision8: 0.3846 - val_precision9: 0.6000 - val_precision10: 0.0000e+00 - val_recall0: 0.0000e+00 - val_recall1: 0.0930 - val_recall2: 0.4694 - val

In [18]:
history275e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_275')
history_array.append(history275e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_275.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #275 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 91s 2s/step - loss: 1.7055 - accuracy: 0.3966 - precision0: 0.6667 - precision1: 0.6970 - precision2: 0.5402 - precision3: 0.5185 - precision4: 0.6667 - precision5: 0.5000 - precision6: 0.7848 - precision7: 0.5455 - precision8: 0.4571 - precision9: 0.5798 - precision10: 0.3333 - recall0: 0.0368 - recall1: 0.1756 - recall2: 0.2814 - recall3: 0.1474 - recall4: 0.0211 - recall5: 0.0330 - recall6: 0.4189 - recall7: 0.0484 - recall8: 0.0833 - recall9: 0.2851 - recall10: 0.0071 - roc_auc: 0.8280 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.4786 - val_accuracy: 0.5010 - val_precision0: 1.0000 - val_precision1: 0.7778 - val_precision2: 0.7436 - val_precision3: 0.3846 - val_precision4: 0.6667 - val_precision5: 1.0000 - val_precision6: 0.8621 - val_precision7: 0.2500 - val_precision8: 0.6000 - val_precision9: 0.6406 - val_precision10: 0.0000e+00 - val_recall0: 0.1000 - val_recall1: 0.1892 - val_recall2: 0.4833 - val_recall3: 0.1111 - v

In [19]:
history300e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_300')
history_array.append(history300e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_300.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #300 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 84s 1s/step - loss: 1.6787 - accuracy: 0.4090 - precision0: 0.6667 - precision1: 0.6129 - precision2: 0.6067 - precision3: 0.5077 - precision4: 0.6250 - precision5: 0.7059 - precision6: 0.7534 - precision7: 0.5000 - precision8: 0.6304 - precision9: 0.6452 - precision10: 0.2500 - recall0: 0.0488 - recall1: 0.1624 - recall2: 0.2951 - recall3: 0.1737 - recall4: 0.0568 - recall5: 0.0622 - recall6: 0.4264 - recall7: 0.0444 - recall8: 0.1429 - recall9: 0.3361 - recall10: 0.0074 - roc_auc: 0.8296 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.5331 - val_accuracy: 0.4648 - val_precision0: 0.6667 - val_precision1: 0.8571 - val_precision2: 0.6471 - val_precision3: 0.4211 - val_precision4: 0.7500 - val_precision5: 0.6667 - val_precision6: 0.6875 - val_precision7: 0.5000 - val_precision8: 0.6333 - val_precision9: 0.6122 - val_precision10: 0.0000e+00 - val_recall0: 0.0426 - val_recall1: 0.1333 - val_recall2: 0.4314 - val_recall3: 0.1905 - v

In [20]:
history325e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_325')
history_array.append(history325e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_325.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #325 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 85s 1s/step - loss: 1.6364 - accuracy: 0.4338 - precision0: 0.5455 - precision1: 0.6341 - precision2: 0.7595 - precision3: 0.5806 - precision4: 0.5000 - precision5: 0.7500 - precision6: 0.8125 - precision7: 0.6667 - precision8: 0.5472 - precision9: 0.6618 - precision10: 0.0000e+00 - recall0: 0.0400 - recall1: 0.2031 - recall2: 0.3226 - recall3: 0.1925 - recall4: 0.0196 - recall5: 0.1182 - recall6: 0.5159 - recall7: 0.0476 - recall8: 0.1518 - recall9: 0.3734 - recall10: 0.0000e+00 - roc_auc: 0.8401 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.4834 - val_accuracy: 0.4707 - val_precision0: 0.7500 - val_precision1: 0.9091 - val_precision2: 0.6667 - val_precision3: 0.5238 - val_precision4: 1.0000 - val_precision5: 0.8000 - val_precision6: 0.7273 - val_precision7: 0.5455 - val_precision8: 0.6818 - val_precision9: 0.5370 - val_precision10: 0.0000e+00 - val_recall0: 0.0714 - val_recall1: 0.2174 - val_recall2: 0.4898 - val_recall3: 0.

In [21]:
history350e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_350')
history_array.append(history350e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_350.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #350 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 86s 2s/step - loss: 1.6304 - accuracy: 0.4406 - precision0: 0.4545 - precision1: 0.7692 - precision2: 0.6456 - precision3: 0.5821 - precision4: 0.7000 - precision5: 0.8214 - precision6: 0.7647 - precision7: 0.5882 - precision8: 0.5323 - precision9: 0.6116 - precision10: 0.2000 - recall0: 0.0312 - recall1: 0.1481 - recall2: 0.3054 - recall3: 0.2063 - recall4: 0.0700 - recall5: 0.1106 - recall6: 0.5000 - recall7: 0.0758 - recall8: 0.1746 - recall9: 0.3203 - recall10: 0.0075 - roc_auc: 0.8426 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.3787 - val_accuracy: 0.5098 - val_precision0: 0.6667 - val_precision1: 1.0000 - val_precision2: 0.7838 - val_precision3: 0.6286 - val_precision4: 1.0000 - val_precision5: 0.6667 - val_precision6: 0.7778 - val_precision7: 0.6667 - val_precision8: 0.6765 - val_precision9: 0.6585 - val_precision10: 1.0000 - val_recall0: 0.0408 - val_recall1: 0.3056 - val_recall2: 0.5472 - val_recall3: 0.4074 - val_r

In [22]:
history375e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_375')
history_array.append(history375e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_375.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #375 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 85s 1s/step - loss: 1.5960 - accuracy: 0.4445 - precision0: 0.4375 - precision1: 0.7273 - precision2: 0.7126 - precision3: 0.5930 - precision4: 0.6364 - precision5: 0.8000 - precision6: 0.7848 - precision7: 0.6000 - precision8: 0.4821 - precision9: 0.5899 - precision10: 0.2000 - recall0: 0.0493 - recall1: 0.2319 - recall2: 0.3924 - recall3: 0.2537 - recall4: 0.0737 - recall5: 0.0769 - recall6: 0.4769 - recall7: 0.1145 - recall8: 0.1444 - recall9: 0.3402 - recall10: 0.0069 - roc_auc: 0.8497 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.4161 - val_accuracy: 0.4941 - val_precision0: 1.0000 - val_precision1: 0.8125 - val_precision2: 0.7568 - val_precision3: 0.6333 - val_precision4: 0.0000e+00 - val_precision5: 0.7143 - val_precision6: 0.8571 - val_precision7: 0.6364 - val_precision8: 0.5714 - val_precision9: 0.7429 - val_precision10: 1.0000 - val_recall0: 0.0233 - val_recall1: 0.3023 - val_recall2: 0.5091 - val_recall3: 0.3333 - v

In [23]:
history400e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_400')
history_array.append(history400e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_400.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #400 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 82s 1s/step - loss: 1.5353 - accuracy: 0.4621 - precision0: 0.7222 - precision1: 0.6829 - precision2: 0.7143 - precision3: 0.6190 - precision4: 0.4286 - precision5: 0.6471 - precision6: 0.7963 - precision7: 0.7500 - precision8: 0.5974 - precision9: 0.5859 - precision10: 0.7500 - recall0: 0.0812 - recall1: 0.2171 - recall2: 0.3736 - recall3: 0.1444 - recall4: 0.0323 - recall5: 0.1043 - recall6: 0.6014 - recall7: 0.1385 - recall8: 0.2396 - recall9: 0.3086 - recall10: 0.0438 - roc_auc: 0.8624 - Multilabel_confusion_matrix: 448.0000 - val_loss: 1.3473 - val_accuracy: 0.5455 - val_precision0: 0.0000e+00 - val_precision1: 0.5556 - val_precision2: 0.8529 - val_precision3: 0.5556 - val_precision4: 1.0000 - val_precision5: 0.5714 - val_precision6: 0.8974 - val_precision7: 0.8182 - val_precision8: 0.6000 - val_precision9: 0.7407 - val_precision10: 1.0000 - val_recall0: 0.0000e+00 - val_recall1: 0.1724 - val_recall2: 0.6591 - val_recall3: 0.2632

In [24]:
history425e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_425')
history_array.append(history425e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_425.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #425 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 91s 2s/step - loss: 1.5070 - accuracy: 0.4592 - precision0: 0.4500 - precision1: 0.7714 - precision2: 0.7653 - precision3: 0.5797 - precision4: 0.5000 - precision5: 0.6857 - precision6: 0.8000 - precision7: 0.5556 - precision8: 0.5733 - precision9: 0.6250 - precision10: 0.3333 - recall0: 0.0600 - recall1: 0.2000 - recall2: 0.4054 - recall3: 0.2222 - recall4: 0.0860 - recall5: 0.1231 - recall6: 0.5926 - recall7: 0.1316 - recall8: 0.2183 - recall9: 0.3320 - recall10: 0.0074 - roc_auc: 0.8674 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.3425 - val_accuracy: 0.5391 - val_precision0: 0.5000 - val_precision1: 0.9231 - val_precision2: 0.7609 - val_precision3: 0.6111 - val_precision4: 0.8333 - val_precision5: 0.8571 - val_precision6: 0.8537 - val_precision7: 0.6000 - val_precision8: 0.5135 - val_precision9: 0.6087 - val_precision10: 1.0000 - val_recall0: 0.0682 - val_recall1: 0.3000 - val_recall2: 0.6250 - val_recall3: 0.2157 - val_r

In [25]:
history450e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_450')
history_array.append(history450e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_450.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #450 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 82s 1s/step - loss: 1.5202 - accuracy: 0.4660 - precision0: 0.5714 - precision1: 0.6905 - precision2: 0.6700 - precision3: 0.5682 - precision4: 0.6667 - precision5: 0.6579 - precision6: 0.7966 - precision7: 0.6774 - precision8: 0.5890 - precision9: 0.6385 - precision10: 0.7143 - recall0: 0.0526 - recall1: 0.2086 - recall2: 0.4187 - recall3: 0.2703 - recall4: 0.0388 - recall5: 0.1208 - recall6: 0.6225 - recall7: 0.1680 - recall8: 0.2251 - recall9: 0.3416 - recall10: 0.0368 - roc_auc: 0.8657 - Multilabel_confusion_matrix: 448.0000 - val_loss: 1.3432 - val_accuracy: 0.5152 - val_precision0: 0.8333 - val_precision1: 0.9231 - val_precision2: 0.7879 - val_precision3: 0.5172 - val_precision4: 1.0000 - val_precision5: 0.2500 - val_precision6: 0.7907 - val_precision7: 0.8571 - val_precision8: 0.5526 - val_precision9: 0.6735 - val_precision10: 0.5000 - val_recall0: 0.0962 - val_recall1: 0.3243 - val_recall2: 0.5532 - val_recall3: 0.3659 - val_r

In [26]:
history475e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_475')
history_array.append(history475e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_475.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #475 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 97s 2s/step - loss: 1.5039 - accuracy: 0.4637 - precision0: 0.5833 - precision1: 0.7500 - precision2: 0.6991 - precision3: 0.6042 - precision4: 0.6667 - precision5: 0.6667 - precision6: 0.8542 - precision7: 0.7778 - precision8: 0.5625 - precision9: 0.6761 - precision10: 0.3333 - recall0: 0.0921 - recall1: 0.2093 - recall2: 0.4731 - recall3: 0.3085 - recall4: 0.0625 - recall5: 0.1128 - recall6: 0.5899 - recall7: 0.1721 - recall8: 0.2250 - recall9: 0.3887 - recall10: 0.0143 - roc_auc: 0.8683 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.2438 - val_accuracy: 0.5664 - val_precision0: 1.0000 - val_precision1: 0.7273 - val_precision2: 0.7660 - val_precision3: 0.6757 - val_precision4: 0.9000 - val_precision5: 0.7778 - val_precision6: 0.8605 - val_precision7: 0.6667 - val_precision8: 0.5957 - val_precision9: 0.6939 - val_precision10: 1.0000 - val_recall0: 0.0426 - val_recall1: 0.2051 - val_recall2: 0.6792 - val_recall3: 0.4717 - val_r

In [27]:
history500e = model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
model.save_weights('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/weights/epochs_500')
history_array.append(history500e.history)
combined_history = combine_multiple_histories(history_array)
with open('D:/y3/licenta_shit/new_experiments/efficientnetv2b0/cce_32_0_00001/history/epochs_500.json', 'wb') as file:
    dump(combined_history, file)

test_results = model.evaluate(test_dataset)
test_precision = get_precision_evaluation(test_results)
test_recall = get_recall_evaluation(test_results)
test_f1_score = get_f1_score(test_precision, test_recall)
test_roc_auc_index = 2 + (2 * num_classes)
    
print("\n\n########################### ACCURACY RESULTS FOR EPOCH #500 ##############################")
print("Test loss: {}".format(test_results[0]))
print("Test accuracy: {}".format(test_results[1]))
print("Test precision: {}".format(test_precision))
print("Test recall: {}".format(test_recall))
print("Test F1 score: {}".format(test_f1_score))
print("Test ROC: {}".format(test_results[test_roc_auc_index]))

Epoch 1/25
56/56 [==============================] - 85s 1s/step - loss: 1.4791 - accuracy: 0.4879 - precision0: 0.5556 - precision1: 0.7308 - precision2: 0.7426 - precision3: 0.6184 - precision4: 0.6667 - precision5: 0.5682 - precision6: 0.7658 - precision7: 0.7000 - precision8: 0.6444 - precision9: 0.6779 - precision10: 0.7000 - recall0: 0.0671 - recall1: 0.3193 - recall2: 0.4286 - recall3: 0.2554 - recall4: 0.1359 - recall5: 0.1316 - recall6: 0.5944 - recall7: 0.1736 - recall8: 0.2843 - recall9: 0.4208 - recall10: 0.0476 - roc_auc: 0.8717 - Multilabel_confusion_matrix: 443.7500 - val_loss: 1.2706 - val_accuracy: 0.5488 - val_precision0: 1.0000 - val_precision1: 0.7500 - val_precision2: 0.7273 - val_precision3: 0.6250 - val_precision4: 0.7143 - val_precision5: 1.0000 - val_precision6: 0.8182 - val_precision7: 0.7273 - val_precision8: 0.5167 - val_precision9: 0.5818 - val_precision10: 1.0000 - val_recall0: 0.1739 - val_recall1: 0.2647 - val_recall2: 0.6486 - val_recall3: 0.2632 - val_r

In [28]:
len(history25e.history['loss'])

25

In [29]:
len(combined_history['loss'])

500